In [15]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import textwrap
import pandas as pd
from geojson import Feature, FeatureCollection, Point

In [16]:
df = pd.read_csv('RussianOligarchPropertyMap.csv')

In [17]:
df.columns

Index(['Still Own?', 'Sale Date', 'Name', 'Oligarch_Description',
       'Full_Address', 'City', 'Sale Price', 'Property Description',
       'Property Type', 'Story/Reference Link', 'Story/Reference Link #2'],
      dtype='object')

In [18]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [ ]:
df['loc'] = df['Full_Address'].apply(geolocator.geocode, timeout=10)

In [ ]:
df['loc']

In [ ]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [ ]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [ ]:
#df.at[3,'building_name']=('87 Park')

In [ ]:
df.columns

In [ ]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=4)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Russian Oligarch Real Estate Transactions/investments')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    desc = r['Oligarch_Description']
    splitted = textwrap.wrap(desc, 50)
    desc = '<br>'.join(splitted)
    info = (r['Name'],r['Full_Address'],desc)
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Name: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Brief Bio: \
    {bold_start}{info[2]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))

m

In [ ]:
m.save('index.html')